# Semantic Search with Amazon OpenSearch 

This is quick demo on how to use Amazon OpeSearch develop semantic search application.

![word vector](word2vec.png)


## Step 1: Prepare BERT Model in SageMaker

### Upgrade PyTorch and restart Kernel

In [1]:
!pip install --upgrade torch

     |████████████████████████████████| 881.9 MB 6.8 kB/s              
  Attempting uninstall: torch
    Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 1.0.61 requires nvidia-ml-py3, which is not installed.


In [2]:
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
restartkernel()

### Verify PyTorch version

In [1]:
import torch
print(torch.__version__)

1.10.2+cu102


### Install required libarary, such as HuggingFace

In [2]:
!pip install -q transformers
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q opensearch-py
!pip install -q tqdm
!pip install -q install transformers[torch]

### Print SageMaker version

In [3]:
import boto3
import re
import time
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()

s3_resource = boto3.resource("s3")
s3 = boto3.client('s3')

print(f'SageMaker SDK Version: {sagemaker.__version__}')

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


SageMaker SDK Version: 2.106.0


In [5]:
cfn = boto3.client('cloudformation')

def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## Setup variables to use for the rest of the demo
cloudformation_stack_name = "semantic-search"

outputs = get_cfn_outputs(cloudformation_stack_name)

bucket = outputs['s3BucketTraining']
aos_host = outputs['DomainEndpoint']

outputs

{'DomainEndpoint': 'search-opensearchservi-yomfx3pcjpau-q4udafmcuza4w5fgpyea4xyvci.us-east-1.es.amazonaws.com',
 'S3BucketSecureURL': 'https://semantic-search-2-s3buckethosting-33xm5021tpid.s3.amazonaws.com',
 'SageMakerNotebookURL': 'https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/notebook-instances/openNotebook/NotebookInstance-kj0OB9Muj8Ed?view=classic',
 's3BucketTraining': 'semantic-search-2-s3buckettraining-8dfzorcextw5',
 'Arn': 'arn:aws:es:us-east-1:522880334446:domain/opensearchservi-yomfx3pcjpau',
 'osDomainName': 'opensearchservi-yomfx3pcjpau',
 's3BucketHostingBucketName': 'semantic-search-2-s3buckethosting-33xm5021tpid'}

Use Hugging Face BERT model to generate vectorization data, every sentence is 768 dimention data.
![BERT](nlp_bert.png)

In [8]:
import torch
from transformers import AutoTokenizer, AutoModel
from transformers import DistilBertTokenizer, DistilBertModel

#model_name = "distilbert-base-uncased"
#model_name = "sentence-transformers/msmarco-distilbert-base-dot-prod-v3"
model_name = "sentence-transformers/distilbert-base-nli-stsb-mean-tokens"


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


def sentence_to_vector(raw_inputs):
    tokenizer = DistilBertTokenizer.from_pretrained(model_name)
    model = DistilBertModel.from_pretrained(model_name)
    inputs_tokens = tokenizer(raw_inputs, padding=True, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model(**inputs_tokens)

    sentence_embeddings = mean_pooling(outputs, inputs_tokens['attention_mask'])
    return sentence_embeddings


### Save pre-trained BERT model to local and then upload to S3

In this section will host the pretrained BERT model into SageMaker Pytorch model server to generate 768x1 dimension fixed length sentence embedding from [sentence-transformers](https://github.com/UKPLab/sentence-transformers) using [HuggingFace Transformers](https://huggingface.co/sentence-transformers/distilbert-base-nli-stsb-mean-tokens). 


In [9]:
import os
from transformers import AutoTokenizer, AutoModel
saved_model_dir = 'transformer'
os.makedirs(saved_model_dir, exist_ok=True)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name) 

tokenizer.save_pretrained(saved_model_dir)
model.save_pretrained(saved_model_dir)

Downloading:   0%|          | 0.00/505 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/555 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/253M [00:00<?, ?B/s]

In [10]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()


In [11]:
!cd transformer && tar czvf ../model.tar.gz *

config.json
pytorch_model.bin
special_tokens_map.json
tokenizer_config.json
tokenizer.json
vocab.txt


In [12]:
#Upload the model to S3

inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='sentence-transformers-model')
inputs

's3://sagemaker-us-east-1-522880334446/sentence-transformers-model/model.tar.gz'

### Deploy the BERT model to SageMaker Endpoint

First we need to create a PyTorchModel object. The deploy() method on the model object creates an endpoint which serves prediction requests in real-time. If the instance_type is set to a SageMaker instance type (e.g. ml.m5.large) then the model will be deployed on SageMaker. If the instance_type parameter is set to local then it will be deployed locally as a Docker container and ready for testing locally.

First we need to create a Predictor class to accept TEXT as input and output JSON. The default behaviour is to accept a numpy array.

In [13]:
from sagemaker.pytorch import PyTorch, PyTorchModel
from sagemaker.predictor import Predictor
from sagemaker import get_execution_role

class StringPredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

Deploy the BERT model to Sagemaker Endpoint

#### Note: This process will take serveral minutes to complete.

In [14]:
pytorch_model = PyTorchModel(model_data = inputs, 
                             role=role, 
                             entry_point ='inference.py',
                             source_dir = './code',
                             py_version = 'py38', 
                             framework_version = '1.10.2',
                             predictor_cls=StringPredictor)

predictor = pytorch_model.deploy(instance_type='ml.g4dn.xlarge', 
                                 initial_instance_count=1, 
                                 endpoint_name = f'semantic-search-model-{int(time.time())}')

----------!

content_type is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### Test the SageMaker Endpoint.

Input is text data, output is vector data

In [15]:
import json
original_payload = 'Does this work with xbox?'
features = predictor.predict(original_payload)
vector_data = json.loads(features)

vector_data

[-0.0703728199005127,
 0.12363174557685852,
 -0.17775849997997284,
 0.3785744905471802,
 0.10541858524084091,
 -0.17814600467681885,
 1.1218693256378174,
 -0.18182483315467834,
 0.8721064925193787,
 -0.07338858395814896,
 -0.07336749136447906,
 0.329407662153244,
 -0.7744451761245728,
 -0.27920395135879517,
 0.16999462246894836,
 -0.9044399857521057,
 -1.142138123512268,
 0.5440396666526794,
 0.1448758840560913,
 -0.06362175196409225,
 -0.0671938806772232,
 0.2200492024421692,
 -0.3382805585861206,
 1.225293755531311,
 -0.08813898265361786,
 0.4586910307407379,
 1.046619176864624,
 0.02334742806851864,
 -0.008586540818214417,
 0.7446296215057373,
 -0.03727012127637863,
 0.3965935707092285,
 -0.6779798865318298,
 0.7318074703216553,
 0.16607773303985596,
 0.16313430666923523,
 0.1956118941307068,
 1.6662781238555908,
 1.0857691764831543,
 -0.7747547626495361,
 0.18122997879981995,
 -0.06501813232898712,
 0.9792707562446594,
 0.45116180181503296,
 0.34111225605010986,
 0.2216665297746658

## Step 2: Ingest data to OpenSearch Cluster
Load data set of Amazon Product Question and Answer data from : https://registry.opendata.aws/amazon-pqa/

### Downloading Amazon Production Question and Answer Data

Datasets: https://registry.opendata.aws/amazon-pqa/

In [17]:
!aws s3 ls --no-sign-request s3://amazon-pqa/

2021-05-20 13:11:25 2267692311 amazon-pqa.tar.gz
2021-05-09 11:53:53  442066567 amazon_pqa_accessories.json
2021-05-09 11:53:49  275062405 amazon_pqa_activity_&_fitness_trackers.json
2021-05-09 11:53:49  127094083 amazon_pqa_adapters.json
2021-05-09 11:53:49  143639699 amazon_pqa_amazon_echo_&_alexa_devices.json
2021-05-09 11:53:49  106017252 amazon_pqa_area_rugs.json
2021-05-09 11:53:49  164430689 amazon_pqa_backpacks.json
2021-05-09 11:53:49  679285046 amazon_pqa_basic_cases.json
2021-05-09 11:53:49  390964941 amazon_pqa_batteries.json
2021-05-09 11:53:49  107896488 amazon_pqa_battery_chargers.json
2021-05-09 11:53:49   77113272 amazon_pqa_bed_frames.json
2021-05-09 11:53:49  157944761 amazon_pqa_beds.json
2021-05-09 11:53:49  218133567 amazon_pqa_bullet_cameras.json
2021-05-09 11:53:50  118106256 amazon_pqa_camcorders.json
2021-05-09 11:53:50   71239417 amazon_pqa_car.json
2021-05-09 11:53:50  137487049 amazon_pqa_car_stereo_receivers.json
2021-05-09 11:53:50  153301436 amazon_pqa_c

In [18]:
!aws s3 cp --no-sign-request s3://amazon-pqa/amazon_pqa_headsets.json ./amazon-pqa/amazon_pqa_headsets.json

download: s3://amazon-pqa/amazon_pqa_headsets.json to amazon-pqa/amazon_pqa_headsets.json


### We can ingest 1000 rows data for test

In [19]:
import json
import pandas as pd

def load_pqa(file_name,number_rows=1000):
    qa_list = []
    df = pd.DataFrame(columns=('question', 'answer'))
    with open(file_name) as f:
        i=0
        for line in f:
            data = json.loads(line)
            df.loc[i] = [data['question_text'],data['answers'][0]['answer_text']]
            i+=1
            if(i == number_rows):
                break
    return df


qa_list = load_pqa('amazon-pqa/amazon_pqa_headsets.json',number_rows=1000)




Convert the text data into vector data

In [20]:
vector_sentences = sentence_to_vector(qa_list["question"].tolist())

Use Python API to set up connection with OpenSearch Cluster

In [21]:
# from elasticsearch import Elasticsearch, RequestsHttpConnection
# from requests_aws4auth import AWS4Auth
# region = 'us-east-1' 
# service = 'es'
# credentials = boto3.Session().get_credentials()
# awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)

# es_client = Elasticsearch(
#     hosts = [{'host': aos_host, 'port': 443}],
#     http_auth = awsauth,
#     use_ssl = True,
#     verify_certs = True,
#     connection_class = RequestsHttpConnection
# )

from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import boto3

#es_host = 'search-semanti-domain-7fc1mmzarfpg-vtklyjm33bhijjarsdhbyl7jxq.us-east-1.es.amazonaws.com' 
region = 'us-east-1' 

credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region)
index_name = 'nlp_pqa'

aos_client = OpenSearch(
    hosts = [{'host': aos_host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

Create a index with 2 fields, the first field is "content" for raw sentece, the second field is "nlp_article_vector" for vector data.

In [22]:
knn_index = {
    "settings": {
        "index.knn": True,
        "index.knn.space_type": "cosinesimil",
        "analysis": {
          "analyzer": {
            "default": {
              "type": "standard",
              "stopwords": "_english_"
            }
          }
        }
    },
    "mappings": {
        "properties": {
            "question_vector": {
                "type": "knn_vector",
                "dimension": 768,
                "store": True
            },
            "question": {
                "type": "text",
                "store": True
            },
            "answer": {
                "type": "text",
                "store": True
            }
        }
    }
}


In [39]:
aos_client.indices.delete(index="nlp_pqa")


{'acknowledged': True}

In [40]:
aos_client.indices.create(index="nlp_pqa",body=knn_index,ignore=400)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'nlp_pqa'}

Show the created index information

In [41]:
aos_client.indices.get(index="nlp_pqa")

{'nlp_pqa': {'aliases': {},
  'mappings': {'properties': {'answer': {'type': 'text', 'store': True},
    'question': {'type': 'text', 'store': True},
    'question_vector': {'type': 'knn_vector',
     'store': True,
     'dimension': 768}}},
  'settings': {'index': {'number_of_shards': '5',
    'provided_name': 'nlp_pqa',
    'knn.space_type': 'cosinesimil',
    'knn': 'true',
    'creation_date': '1663212998258',
    'analysis': {'analyzer': {'default': {'type': 'standard',
       'stopwords': '_english_'}}},
    'number_of_replicas': '1',
    'uuid': '_fYL12HqRrSrf1D8qQqF7Q',
    'version': {'created': '135248027'}}}}}

In [42]:
i = 0
for c in qa_list["question"].tolist():
    content=c
    vector=vector_sentences[i].tolist()
    answer=qa_list["answer"][i]
    i+=1
    aos_client.index(index='nlp_pqa',body={"question_vector": vector, "question": content,"answer":answer})

### Ingest all the headset PQA data into OpenSearch Cluster
Comment out the following code to ingest all the headset question, answer and corresponding question vector data into OpenSearch index. 

### Note: it will take more than 10 minutes to complete.

In [43]:
# import json
# from tqdm.contrib.concurrent import process_map
# from multiprocessing import cpu_count


# def load_pqa_as_json(file_name):
#     result=[]
#     with open(file_name) as f:
#         for line in f:
#             data = json.loads(line)
#             result.append(data)
#     return result


# qa_list_json = load_pqa_as_json('amazon-pqa/amazon_pqa_headsets.json')


# def es_import(question):
#     vector = json.loads(predictor.predict(question["question_text"]))
#     aos_client.index(index='nlp_pqa',
#              body={"question_vector": vector, "question": question["question_text"],"answer":question["answers"][0]["answer_text"]}
#             )
        
# workers = 4 * cpu_count()
    
# process_map(es_import, qa_list_json, max_workers=workers,chunksize=1000)

### Query the documents number in the OpenSearch Cluster

In [44]:
res = aos_client.search(index="nlp_pqa", body={"query": {"match_all": {}}})
print("Got %d Hits:" % res['hits']['total']['value'])

Got 1000 Hits:


## Step 3: Semantic Search 
### Generate vector data for user input query 

Generate vector data for the question by calling SageMaker model

In [45]:
query_raw_sentences = ['does this work with xbox?']
client = boto3.client('sagemaker-runtime')
ENDPOINT_NAME = predictor.endpoint
response = client.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                       ContentType='text/plain',
                                       Body=query_raw_sentences[0])

search_vector = json.loads((response['Body'].read()))


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### Search vector data with "Semanatic Search" 

OpenSearch KNN


In [46]:

query={
    "size": 50,
    "query": {
        "knn": {
            "question_vector":{
                "vector":search_vector,
                "k":50
            }
        }
    }
}

res = aos_client.search(index="nlp_pqa", 
                       body=query,
                       stored_fields=["question","answer"])
#print("Got %d Hits:" % res['hits']['total']['value'])
query_result=[]
for hit in res['hits']['hits']:
    row=[hit['_id'],hit['_score'],hit['fields']['question'][0],hit['fields']['answer'][0]]
    query_result.append(row)

query_result_df = pd.DataFrame(data=query_result,columns=["_id","_score","question","answer"])
display(query_result_df)

,_id,_score,question,answer
0,bz84P4MB3Pe6ZRwqboIp,0.976584,Does this work with xbox one?,"sorry, Im not an xbox user."
1,ez84P4MB3Pe6ZRwqhoTV,0.976127,Does this work with the xbox one?,"Yeah of course , but you must have an adapter ..."
2,7T84P4MB3Pe6ZRwqgIMl,0.967617,does this work on xbox one?,"I'm sorry, but not!"
3,5T84P4MB3Pe6ZRwqZ4Hj,0.966998,Does this work for xbox one S?,It should work.
4,_T84P4MB3Pe6ZRwqgIPg,0.963552,Does it work for xbox one?,"Thanks for your inquiry, it just works with PS..."
5,WD84P4MB3Pe6ZRwqkYUp,0.954933,Will it work with Xbox One?,"With the chat adapter for xbox one remotes, bi..."
6,8T84P4MB3Pe6ZRwqaIFy,0.953493,Do they work with xbox one?,"No they don't , but let's hope that Microsoft ..."
7,zD84P4MB3Pe6ZRwqfoOg,0.952881,will these work with xbox one?,Yes
8,Dz84P4MB3Pe6ZRwqgYS1,0.950155,Will it work for xbox one?,"Sorry, it is not compatible with PS4 Xbox one...."
9,dD84P4MB3Pe6ZRwqkoVr,0.948001,Do they work with Xbox One system?,If you have the Xbox controller stereo headset...


### Search the same query with "Keyword Search"

In [47]:
query={
    "size": 50,
    "query": {
        "match": {
            "question":"does this work with xbox?"
        }
    }
}

res = aos_client.search(index="nlp_pqa", 
                       body=query,
                       stored_fields=["question","answer"])
#print("Got %d Hits:" % res['hits']['total']['value'])
query_result=[]
for hit in res['hits']['hits']:
    row=[hit['_id'],hit['_score'],hit['fields']['question'][0],hit['fields']['answer'][0]]
    query_result.append(row)

query_result_df = pd.DataFrame(data=query_result,columns=["_id","_score","question","answer"])
display(query_result_df)


,_id,_score,question,answer
0,bz84P4MB3Pe6ZRwqboIp,7.566119,Does this work with xbox one?,"sorry, Im not an xbox user."
1,9j84P4MB3Pe6ZRwqgIOQ,7.003384,Does it work for Xbox 360?,"Sorry , it can't .Just for PS4"
2,FT84P4MB3Pe6ZRwqaoII,7.003384,does it work for an xbox 1?,As long as your controller has a 3.5 headset j...
3,ez84P4MB3Pe6ZRwqhoTV,6.853209,Does this work with the xbox one?,"Yeah of course , but you must have an adapter ..."
4,7T84P4MB3Pe6ZRwqgIMl,6.853209,does this work on xbox one?,"I'm sorry, but not!"
5,_T84P4MB3Pe6ZRwqgIPg,6.745397,Does it work for xbox one?,"Thanks for your inquiry, it just works with PS..."
6,Qj84P4MB3Pe6ZRwqj4Xs,6.655265,does it work on xbox 1,I am not sure about that but for the price I w...
7,Kz84P4MB3Pe6ZRwqjoXC,6.440127,Does it work for Xbox 360 and os4,Yes. If you get the correct mixamp for those c...
8,-j84P4MB3Pe6ZRwqgIO-,6.352438,If you have an adaptor does it work for xbox one,Yes it does. Enjoy
9,5T84P4MB3Pe6ZRwqZ4Hj,6.258648,Does this work for xbox one S?,It should work.


## Step 4: Deploying a full-stack semantic search application

### Disable S3 "Block all public access"

Go to S3 Console, click "Block Public Access settings for this account" make sure "Block all public access" is off.

In [49]:
s3_resource.Object(bucket, 'backend/template.yaml').upload_file('./backend/template.yaml', ExtraArgs={'ACL':'public-read'})


sam_template_url = f'https://{bucket}.s3.amazonaws.com/backend/template.yaml'
print("cloudformation template url:" + sam_template_url)


# Generate the CloudFormation Quick Create Link

print("Click the URL below to create the backend API for NLU search:\n")
print((
    'https://console.aws.amazon.com/cloudformation/home?region=us-east-1#/stacks/create/review'
    f'?templateURL={sam_template_url}'
    '&stackName=semantic-search-api'
    f'&param_BucketName={outputs["s3BucketTraining"]}'
    f'&param_DomainName={outputs["osDomainName"]}'
    f'&param_ElasticSearchURL={outputs["DomainEndpoint"]}'
    f'&param_SagemakerEndpoint={predictor.endpoint}'
))

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


cloudformation template url:https://semantic-search-2-s3buckettraining-8dfzorcextw5.s3.amazonaws.com/backend/template.yaml
Click the URL below to create the backend API for NLU search:

https://console.aws.amazon.com/cloudformation/home?region=us-east-1#/stacks/create/review?templateURL=https://semantic-search-2-s3buckettraining-8dfzorcextw5.s3.amazonaws.com/backend/template.yaml&stackName=semantic-search-api&param_BucketName=semantic-search-2-s3buckettraining-8dfzorcextw5&param_DomainName=opensearchservi-yomfx3pcjpau&param_ElasticSearchURL=search-opensearchservi-yomfx3pcjpau-q4udafmcuza4w5fgpyea4xyvci.us-east-1.es.amazonaws.com&param_SagemakerEndpoint=semantic-search-model-1663211529


Now that you have a working Amazon SageMaker endpoint for extracting image features and a KNN index on Elasticsearch, you are ready to build a real-world full-stack ML-powered web app. The SAM template you just created will deploy an Amazon API Gateway and AWS Lambda function. The Lambda function runs your code in response to HTTP requests that are sent to the API Gateway.

In [50]:
!pygmentize backend/lambda/app.py

import json
from os import environ

import boto3
from urllib.parse import urlparse

from elasticsearch import Elasticsearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth

# Global variables that are reused
sm_runtime_client = boto3.client('sagemaker-runtime')
s3_client = boto3.client('s3')


def get_features(sm_runtime_client, sagemaker_endpoint, payload):
    response = sm_runtime_client.invoke_endpoint(
        EndpointName=sagemaker_endpoint,
        ContentType='text/plain',
        Body=payload)
    response_body = json.loads((response['Body'].read()))
    features = response_body

    return features


def get_neighbors(features, es, k_neighbors=50):
    idx_name = 'nlp_pqa'
    res = es.search(
        request_timeout=30, index=idx_name,
        body={
            'size': k_neighbors,
            'query': {'knn': {'question_vector': {'vector': features, 'k': k_neighbors}}}},
        stored_fields=["question","answer"]
        )
    results = [{'question':res['hi

## Once the CloudFormation Stack shows CREATE_COMPLETE, proceed to this cell below:

In [51]:
import json
api_endpoint = get_cfn_outputs('semantic-search-api')['TextSimilarityApi']

with open('./frontend/src/config/config.json', 'w') as outfile:
    json.dump({'apiEndpoint': api_endpoint}, outfile)

## Deploy frontend services

In [52]:
# add NPM to the path so we can assemble the web frontend from our notebook code

from os import environ

npm_path = ':/home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin'

if npm_path not in environ['PATH']:
    ADD_NPM_PATH = environ['PATH']
    ADD_NPM_PATH = ADD_NPM_PATH + npm_path
else:
    ADD_NPM_PATH = environ['PATH']
    
%set_env PATH=$ADD_NPM_PATH

env: PATH=/usr/local/cuda-10.1/bin:/opt/amazon/openmpi/bin:/opt/amazon/efa/bin:/home/ec2-user/anaconda3/condabin:/home/ec2-user/.dl_binaries/bin:/usr/local/cuda/bin:/usr/libexec/gcc/x86_64-amazon-linux/4.8.5:/home/ec2-user/anaconda3/envs/pytorch_p36/bin:/home/ec2-user/anaconda3/condabin:/opt/amazon/openmpi/bin:/opt/amazon/efa/bin:/home/ec2-user/anaconda3/condabin:/home/ec2-user/.dl_binaries/bin:/usr/local/cuda/bin:/usr/libexec/gcc/x86_64-amazon-linux/4.8.5:/usr/local/bin:/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/sbin:/opt/aws/bin:/opt/aws/bin:/home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin


In [53]:
%cd ./frontend/

!npm install

/home/ec2-user/SageMaker/semantic-search/frontend
npm WARN read-shrinkwrap This version of npm is compatible with lockfileVersion@1, but package-lock.json was generated for lockfileVersion@2. I'll try to do my best with it!
        ......] - postinstall:@aws-sdk/credential-provider-sso: info lifecm
> core-js@3.25.0 postinstall /home/ec2-user/SageMaker/semantic-search/frontend/node_modules/core-js
> node -e "try{require('./postinstall')}catch(e){}"

Thank you for using core-js ( https://github.com/zloirock/core-js ) for polyfilling JavaScript standard library!

The project needs your help! Please consider supporting of core-js:
> https://opencollective.com/core-js 
> https://patreon.com/zloirock 
> bitcoin: bc1qlea7544qtsmj2rayg0lthvza9fau63ux0fstcz 

Also, the author of core-js ( https://github.com/zloirock ) is looking for a good job -)


> core-js-pure@3.25.0 postinstall /home/ec2-user/SageMaker/semantic-search/frontend/node_modules/core-js-pure
> node -e "try{require('./postinstall'

In [54]:
!npm run-script build


> frontend@0.1.0 build /home/ec2-user/SageMaker/semantic-search/frontend
> react-scripts build

Creating an optimized production build...
Compiled with warnings.

[eslint] 
src/App.js
  Line 4:25:  'FormControl' is defined but never used                    no-unused-vars
  Line 4:38:  'Select' is defined but never used                         no-unused-vars
  Line 4:46:  'MenuItem' is defined but never used                       no-unused-vars
  Line 10:8:  'GridList' is defined but never used                       no-unused-vars
  Line 11:8:  'GridListTile' is defined but never used                   no-unused-vars
  Line 12:8:  'GridListTileBar' is defined but never used                no-unused-vars
  Line 51:7:  'BorderLinearProgress' is assigned a value but never used  no-unused-vars

src/config/index.js
  Line 3:1:  Assign object to a variable before exporting as module default  import/no-anonymous-default-export

Search for the keywords to learn more about each warning.
To igno

In [55]:
hosting_bucket = f"s3://{outputs['s3BucketHostingBucketName']}"

!aws s3 sync ./build/ $hosting_bucket --acl public-read

upload: build/asset-manifest.json to s3://semantic-search-2-s3buckethosting-33xm5021tpid/asset-manifest.json
upload: build/favicon.ico to s3://semantic-search-2-s3buckethosting-33xm5021tpid/favicon.ico
upload: build/static/js/main.2dde8bb0.js.LICENSE.txt to s3://semantic-search-2-s3buckethosting-33xm5021tpid/static/js/main.2dde8bb0.js.LICENSE.txt
upload: build/robots.txt to s3://semantic-search-2-s3buckethosting-33xm5021tpid/robots.txt
upload: build/index.html to s3://semantic-search-2-s3buckethosting-33xm5021tpid/index.html
upload: build/logo192.png to s3://semantic-search-2-s3buckethosting-33xm5021tpid/logo192.png
upload: build/static/css/main.4ef2127a.css.map to s3://semantic-search-2-s3buckethosting-33xm5021tpid/static/css/main.4ef2127a.css.map
upload: build/logo512.png to s3://semantic-search-2-s3buckethosting-33xm5021tpid/logo512.png
upload: build/manifest.json to s3://semantic-search-2-s3buckethosting-33xm5021tpid/manifest.json
upload: build/static/css/main.4ef2127a.css to s3://

## Browse your frontend service

In [56]:
print('Click the URL below:\n')
print(outputs['S3BucketSecureURL'] + '/index.html')

Click the URL below:

https://semantic-search-2-s3buckethosting-33xm5021tpid.s3.amazonaws.com/index.html


## Cleanup

Make sure that you stop the notebook instance, delete the Amazon SageMaker endpoint and delete the Elasticsearch domain to prevent any additional charges.

In [57]:
# Delete the endpoint
predictor.delete_endpoint()

# Empty S3 Contents
training_bucket_resource = s3_resource.Bucket(bucket)
training_bucket_resource.objects.all().delete()

hosting_bucket_resource = s3_resource.Bucket(outputs['s3BucketHostingBucketName'])
hosting_bucket_resource.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'PMH3C444Z3KMYB9P',
   'HostId': 'TRYnjHLLu+lp4uLk8/YT3tggrJNQaC/BHFLG2+n4JyVTikNwmQUFafyXfxjMReM/RSnEvMiBwoU=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'TRYnjHLLu+lp4uLk8/YT3tggrJNQaC/BHFLG2+n4JyVTikNwmQUFafyXfxjMReM/RSnEvMiBwoU=',
    'x-amz-request-id': 'PMH3C444Z3KMYB9P',
    'date': 'Thu, 15 Sep 2022 03:46:04 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'asset-manifest.json',
    'DeleteMarker': True,
    'DeleteMarkerVersionId': 'vv88rksENNmPAKH17FdzlBLWoERZi9fX'},
   {'Key': 'static/media/roboto-latin-900.870c8c1486f76054301a.woff2',
    'DeleteMarker': True,
    'DeleteMarkerVersionId': 'QIDw3Id5K1cBRsyHldO.ha.v0rqQk9VS'},
   {'Key': 'logo512.png',
    'DeleteMarker': True,
    'DeleteMarkerVersionId': 'NH_0RZVBBia0C.ENKiuymNjdJ6ZZVqYh'},
   {'Key': 'static/js/main.2dde8bb0.js.LICENSE.txt',